# RAG APP using Groq API and Langchain

In [20]:
import os 
import getpass
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import warnings 

warnings.filterwarnings("ignore")

In [7]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Langsmith api key")

load an llm

In [8]:
groq_llm = ChatGroq(
                groq_api_key = os.environ["GROQ_API_KEY"],
                model="llama-3.1-8b-instant",
                temperature = 0.6
                )
#test
groq_llm.invoke("Hello").content

load an embedder 

In [ ]:
embedder = HuggingFaceEmbeddings( 
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)
#test
embedder.embed_query("Hello")

Vector store 

In [21]:
embedding_dim = len(embedder.embed_query("test"))

In [23]:
index = faiss.IndexFlatL2(embedding_dim)

In [44]:
vectore_store = FAISS(
    embedding_function = embedder,
    index = index,
    docstore = InMemoryDocstore(),
    index_to_docstore_id = {}
)

In [26]:
vectore_store

RAG

In [80]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain.prompts import PromptTemplate
from typing_extensions import TypedDict, List
from langgraph.graph import StateGraph, START

In [30]:
#load contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [35]:
docs = loader.load()

In [36]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
docs_splitted = splitter.split_documents(docs)

In [45]:
#Add docs to vectore store
_ = vectore_store.add_documents(documents = docs_splitted)

In [61]:
#prompt template
prompt = """
Answer the following question : {question}
using the relevant informations bellow :
{context}
"""
prompt = PromptTemplate(
    template = prompt,
    input_features = ["question", "context"]
    )

In [65]:
#State of application 
class State(TypedDict):
    question : str
    context : List
    answer : str

In [68]:
#test
context = vectore_store.similarity_search("what is sensory memory ?")

In [73]:
context[0].page_content

962

In [67]:
def retrieve (state : State):
    retrieved_docs = vectore_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

In [78]:
def generate(state: State):
    context = "\n\n".join([chunk.page_content for chunk in state["context"]])
    messages = prompt.invoke({"question": state["question"], "context": context})
    response = groq_llm.invoke(messages)
    return {"answer": response.content}

In [82]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [84]:
response = graph.invoke({"question": "What is short term memory capacity ?"})

In [86]:
response["answer"]

'Based on the provided information, the short-term memory capacity is believed to be about 7 items. This is a concept introduced by George Miller in 1956, suggesting that the human brain can hold approximately 7 pieces of information in its short-term memory at any given time.'